In [4]:
import pandas as pd
pd.set_option('display.max_columns', 500)

In [2]:
cards = pd.read_csv("cards.csv")

In [5]:
cards.head()

,Unnamed: 0,atk,base_card_id,card_id,card_name,card_set_id,char_type,clan,copyright,cost,cv,description,evo_atk,evo_description,evo_life,evo_skill_disc,foil_card_id,format_type,get_red_ether,is_foil,life,normal_card_id,org_evo_skill_disc,org_skill_disc,rarity,restricted_count,skill,skill_disc,skill_option,tokens,tribe_name,use_red_ether
0,0,1,900111020,900111020,Fairy Wisp,90000,1,1,NaN,0,NaN,"Gliding upon the wind, as if to illuminate the...",3,"Gliding upon the wind, gently floating as if w...",3,NaN,900111021,1,0,0,1,900111020,NaN,[b]Fanfare[/b][b]:[/b] Banish this follower if...,1,3,banish,Fanfare: Banish this follower if at least 2 ot...,none,NaN,-,0
1,1,0,110124010,110124010,Mark of the Unkilling,10010,4,1,NaN,0,NaN,"Burned by tears or torn by screams, you must b...",0,NaN,0,NaN,110124011,1,50,0,0,110124010,NaN,Banish an enemy follower with 0 attack or less...,2,3,"banish,pp_fixeduse,banish,draw",Banish an enemy follower with 0 attack or less...,"none,fixeduse=6,none,none",NaN,-,200
2,2,0,900144020,900144020,Blessings of Creation,90000,4,1,NaN,0,NaN,"The noble shimmer of life for you, noble one. ...",0,NaN,0,NaN,900144021,1,0,0,0,900144020,NaN,Draw 2 cards.<br>[b]Enhance[/b] [b](2)[/b][b]:...,4,3,"draw,pp_fixeduse,play_count_change",Draw 2 cards.<br>Enhance (2): Add 2 to the num...,"none,fixeduse=2,add_count=2",NaN,-,0
3,3,0,900144030,900144030,Wrath of Nature,90000,4,1,NaN,0,NaN,The noble shimmer of life must not be defiled ...,0,NaN,0,NaN,900144031,1,0,0,0,900144030,NaN,Give an allied follower +1/+0 and [b]Rush[/b]....,4,3,"powerup,rush,pp_fixeduse,quick",Give an allied follower +1/+0 and Rush.<br>Enh...,"add_offense=1,none,fixeduse=4,none",NaN,-,0
4,4,2,900231030,900231030,"Evyn, Novice Agent",90000,1,2,NaN,0,NaN,"Countless traps and snares, and all eyes on th...",2,I can still turn this around! I just gotta sta...,4,Storm.<br>Evolve: Deal 2 damage to an enemy fo...,900231031,1,0,0,2,900231030,[b]Storm[/b][b].[/b]<br>[b]Evolve[/b][b]:[/b] ...,NaN,3,3,"none//quick,damage",NaN,"none//none,damage=2",NaN,Officer,0
